# 1. SETTING
- 환경 설정 및 데이터 URL 불러오기

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
key = '' # 개인정보이므로 삭제했습니다.
myURI = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/TbVwAttractions/1/1000/'

In [ ]:
print(myURI)

# 2. GET OPENAPI

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

def get_openapi(url):
  global df # 전역 변수 설정

  html = urlopen(url)
  bs = BeautifulSoup(html.read(), 'lxml-xml') # html.parser보다 더 깨끗한 결과 제공, 한글 스트링 호환

  itemlist = bs.find_all("row")

  df = pd.DataFrame() # 데이터를 저장할 빈 dataframe 생성

  for t in itemlist:
    try:
      language = t.find('LANG_CODE_ID').get_text()
      name = t.find('POST_SJ').get_text()
      address = t.find('ADDRESS').get_text()
      new_address = t.find('NEW_ADDRESS').get_text()
      tel = t.find('CMMN_TELNO').get_text()
      web_url = t.find('CMMN_HMPG_URL').get_text()
      use_time = t.find('CMMN_USE_TIME').get_text()
      op_info = t.find('CMMN_BSNDE').get_text()
      rest_info = t.find('CMMN_RSTDE').get_text()
      traffic_info = t.find('SUBWAY_INFO').get_text()
      tag = t.find('TAG').get_text()
      disabled_facility = t.find('BF_DESC').get_text()
    
    except: # 제공되지 않는(비어있는) 항목일 경우, 오류없이 ""로 받아오기 
      language = ""
      name = ""
      address = ""
      new_address = ""
      tel = ""
      web_url = ""
      use_time = ""
      op_info = ""
      rest_info = ""
      traffic_info = ""
      tag = ""
      disabled_facility = ""
    
    temp = pd.DataFrame(([[language, name, address, new_address, tel, web_url, use_time, op_info, rest_info, traffic_info, tag, disabled_facility]]), 
                        columns = ["언어", "상호명", "주소", "신주소", "전화번호", "웹페이지", 
                                    "운영시간", "운영요일", "휴무일", "교통정보", "태그", "장애인편의시설"])
    df = pd.concat([df,temp])

  df = df.reset_index(drop=True)
  return df

In [ ]:
get_openapi(myURI)

,언어,상호명,주소,신주소,전화번호,웹페이지,운영시간,운영요일,휴무일,교통정보,태그,장애인편의시설
0,ko,63스퀘어,150-763 서울 영등포구 여의도동 60,"07345 서울 영등포구 63로 50 (여의도동, 63한화생명빌딩)",1833-7001,www.63art.co.kr,매일 10:00 - 19:00 아쿠아리움 / 연중무휴 (운영시간 마감 30분 전 입...,월~일,연중무휴,1호선 대방역 6번 출구 셔틀버스 탑승 \r\n5호선 여의도역 5번 출구 셔틀버...,"여의도, 아쿠아리움, 전망대, 데이트코스, 63아쿠아플라넷, 63스퀘어, 63빌딩레...","엘리베이터,장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실,접근가능"
1,en,Bongeunsa Temple,"135-870 73, Samseong-dong, Gangnam-gu, Seoul, ...","06087 531, Bongeunsa-ro, Gangnam-gu, Seoul, Ko...",+82-2-3218-4800,http://www.bongeunsa.org/,,,,"* Subway Line 9, Bongeunsa Station, Exit 1\r\n...","Gangnam,Coex,HistoricSite,BuddhistTemple,Lante...",Accessible Information Centers &amp; Services(...
2,en,Changdeokgung Palace,"110-360 2-71, Waryong-dong, Jongno-gu, Seoul","03072 2-71, Waryong-dong, Jongno-gu, Seoul",+82-2-762-8261,http://www.cdg.go.kr/eng/,"February - May, 09:00 - 18:00 (Last entry at 1...",,Closed Mondays,"Subway Line 3, Anguk Station, Exit 3 (5 minute...","ChangdeokgungPalace,SeoulHanbokTour,KoreanTrad...",Accessible Information Centers &amp; Services(...
3,en,Changgyeonggung Palace,"110-360 Waryong-dong, Jongno-gu, Seoul 2-1","03072 185, Changgyeonggung-ro, Jongno-gu, Seoul",+82-2-762-4868,http://english.cha.go.kr/html/HtmlPage.do?pg=/...,Daily 09:00 - 21:00 (Last entry at 20:00) *Clo...,,,"Subway Line 4, Hyehwa Station, Exit 4 (15 minu...","Traditional,KoreanHistoricSites,SeoulNightView...",Accessible Information Centers &amp; Services(...
4,en,Culture Station Seoul 284,"100-162 122-28, Bongnae-dong 2(i)-ga, Jung-gu,...","04509 1, Tongil-ro, Jung-gu, Seoul",+82-2-3407-3500,http://www.seoul284.org,Daily 10:00 - 19:00 (Last entry at 18:00)\r\n*...,,"Closed Mondays, New Year, Lunar New Year & Chu...","Subway Lines 1 & 4, Seoul Station, Exit 2","SeoulStation,TrainStation,Exhibition,FormerSeo...",Accessible Information Centers &amp; Services(...
...,...,...,...,...,...,...,...,...,...,...,...,...
192,ko,종묘,110-400 서울 종로구 훈정동 1-2,"03135 서울 종로구 종로 157 (훈정동, 종묘공원)",02-765-0195,http://jm.cha.go.kr/agapp/main/index.do?siteCd=JM,"2월 ~ 5월, 9월 ~ 10월 09:00~18:00 (입장마감 17:00)\r\n...",,화요일,5호선 종로3가역 8번 출구 도보 5분 <br/>\r\n1호선 종로3가역 11번 출...,"창덕궁,종로3가역,종묘제례, 종묘사직,유네스코세계문화유산, 광화문, 종묘, 창경궁","장애인 안내(휠체어 대여 및 기타),장애인화장실,접근가능"
193,ko,창경궁,110-360 서울 종로구 와룡동 2-1,"03072 서울 종로구 창경궁로 185 (와룡동, 창경궁)",02-762-4868,http://cgg.cha.go.kr/agapp/public/html/HtmlPag...,09:00 ~ 21:00 (입장마감 20:00),화~일,월,4호선 혜화역 4번 출구 도보 15분 \r\n,"서울야경,서울야간데이트,서울가볼만한곳,서울궁궐,한국역사적명소,한국궁궐,한국전통문화,...","장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실"
194,ko,창경궁 대온실,서울 종로구 와룡동 2-1,"03072 서울 종로구 창경궁로 185 (와룡동, 창경궁)",(02) 762-4868~9,http://cgg.cha.go.kr/agapp/main/index.do?siteC...,,,,혜화역 4번출구 700m,"창경궁대온실,창경궁데이트,창경궁포토존,서울가볼만한곳,창경궁",장애인 안내(휠체어 대여 및 기타)
195,ko,창덕궁,110-360 서울 종로구 와룡동 2-71,"03072 서울 종로구 율곡로 99 (와룡동, 창덕궁)",02-3668-2300,http://www.cdg.go.kr,2-5월\t09:00 ~ 18:00(입장마감은 17:00)\r\n6-8월\t09:0...,화~일,월,5호선 종로3가역 6번 출구 도보 10분 \r\n3호선 안국역 3번 출구 도보 ...,"창덕궁,서울데이트코스,한국전통,서울필수여행지,서울여행코스,서울가볼만한곳,한국문화,세...","장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실"


# 3. PREPROCESSING
  1. 언어 칼럼을 통해 한국어/영어/일본어/중국어 데이터 가져오기
  2. 언어 칼럼의 데이터를 한글로 대치 (ex. ko -> 한국어)
  3. 언어별 구 추출

In [ ]:
seoul_attraction_df = df

In [ ]:
# 언어 칼럼을 통해 한국어/영어/일본어/중국어 데이터만 가져오기
language = ['ko', 'en', 'ja', 'zh-CN'] # 한국어, 영어, 일본어, 중국어
seoul_attraction_df = seoul_attraction_df[seoul_attraction_df['언어'].isin(language)] # 4개 국어 데이터만 가져오기
seoul_attraction_df = seoul_attraction_df.reset_index(drop=True)

In [ ]:
# 언어에 따라 데이터 대치 (ex. ko -> 한국어)
for i in range(len(seoul_attraction_df)):
  if seoul_attraction_df['언어'][i] == 'ko': # 한국어
    seoul_attraction_df['언어'][i] = '한국어'
  elif seoul_attraction_df['언어'][i] == 'en': # 영어
    seoul_attraction_df['언어'][i] = '영어'
  elif seoul_attraction_df['언어'][i] == 'ja': # 일본어
    seoul_attraction_df['언어'][i] = '일본어'
  else: # 중국어
    seoul_attraction_df['언어'][i] = '중국어'

seoul_attraction_df.head(5)

,언어,상호명,주소,신주소,전화번호,웹페이지,운영시간,운영요일,휴무일,교통정보,태그,장애인편의시설
0,한국어,63스퀘어,150-763 서울 영등포구 여의도동 60,"07345 서울 영등포구 63로 50 (여의도동, 63한화생명빌딩)",1833-7001,www.63art.co.kr,매일 10:00 - 19:00 아쿠아리움 / 연중무휴 (운영시간 마감 30분 전 입...,월~일,연중무휴,1호선 대방역 6번 출구 셔틀버스 탑승 \r\n5호선 여의도역 5번 출구 셔틀버...,"여의도, 아쿠아리움, 전망대, 데이트코스, 63아쿠아플라넷, 63스퀘어, 63빌딩레...","엘리베이터,장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실,접근가능"
1,영어,Bongeunsa Temple,"135-870 73, Samseong-dong, Gangnam-gu, Seoul, ...","06087 531, Bongeunsa-ro, Gangnam-gu, Seoul, Ko...",+82-2-3218-4800,http://www.bongeunsa.org/,,,,"* Subway Line 9, Bongeunsa Station, Exit 1\r\n...","Gangnam,Coex,HistoricSite,BuddhistTemple,Lante...",Accessible Information Centers &amp; Services(...
2,영어,Changdeokgung Palace,"110-360 2-71, Waryong-dong, Jongno-gu, Seoul","03072 2-71, Waryong-dong, Jongno-gu, Seoul",+82-2-762-8261,http://www.cdg.go.kr/eng/,"February - May, 09:00 - 18:00 (Last entry at 1...",,Closed Mondays,"Subway Line 3, Anguk Station, Exit 3 (5 minute...","ChangdeokgungPalace,SeoulHanbokTour,KoreanTrad...",Accessible Information Centers &amp; Services(...
3,영어,Changgyeonggung Palace,"110-360 Waryong-dong, Jongno-gu, Seoul 2-1","03072 185, Changgyeonggung-ro, Jongno-gu, Seoul",+82-2-762-4868,http://english.cha.go.kr/html/HtmlPage.do?pg=/...,Daily 09:00 - 21:00 (Last entry at 20:00) *Clo...,,,"Subway Line 4, Hyehwa Station, Exit 4 (15 minu...","Traditional,KoreanHistoricSites,SeoulNightView...",Accessible Information Centers &amp; Services(...
4,영어,Culture Station Seoul 284,"100-162 122-28, Bongnae-dong 2(i)-ga, Jung-gu,...","04509 1, Tongil-ro, Jung-gu, Seoul",+82-2-3407-3500,http://www.seoul284.org,Daily 10:00 - 19:00 (Last entry at 18:00)\r\n*...,,"Closed Mondays, New Year, Lunar New Year & Chu...","Subway Lines 1 & 4, Seoul Station, Exit 2","SeoulStation,TrainStation,Exhibition,FormerSeo...",Accessible Information Centers &amp; Services(...


In [ ]:
# 주소에서 구 추출
seoul_attraction_df['구'] = ''


for i in range(len(seoul_attraction_df)):
  # '주소' 칼럼을 기준으로 하되, 비어있으면 '신주소' 칼럼을 이용해서 전체 주소 불러오기
  full_address = seoul_attraction_df['주소'][i]
  if full_address == ' ':
    full_address = seoul_attraction_df['신주소'][i]


  if seoul_attraction_df['언어'][i] == '한국어': # 한국어 주소 추출
    try:
      seoul_attraction_df['구'][i] = full_address.split(' ')[2]
      if seoul_attraction_df['구'][i][-1] != '구':
        seoul_attraction_df['구'][i] = 'error'
    except:
      seoul_attraction_df['구'][i] = 'error'

  elif seoul_attraction_df['언어'][i] == '영어': # 영어 주소 추출
    try:
      address = seoul_attraction_df['주소'][i].split(',')
      seoul_attraction_df['구'][i] = address[['-gu' in address[i] for i in range(len(address))].index(True)].replace(' ', '')
    except:
      seoul_attraction_df['구'][i] = 'error'

  elif seoul_attraction_df['언어'][i] == '일본어': # 일본어 주소 추출
    try:
      address1 = full_address.split('市')
      address2 = address1[1].split('区')
      seoul_attraction_df['구'][i] = address2[0]+'区'
    except:
      seoul_attraction_df['구'][i] = 'error'

  else: # 중국어 주소 추출
    if '市' in full_address:
      address1 = full_address.split('市')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_attraction_df['구'][i] = address2[0] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_attraction_df['구'][i] = address2[0] + '区'
      else:
        seoul_attraction_df['구'][i] = 'error'
      
    elif '首' in full_address:
      address1 = full_address.split('首')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_attraction_df['구'][i] = address2[0][1:] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_attraction_df['구'][i] = address2[0][1:] + '区'
      else:
        seoul_attraction_df['구'][i] = 'error'

In [ ]:
seoul_attraction_df

,언어,상호명,주소,신주소,전화번호,웹페이지,운영시간,운영요일,휴무일,교통정보,태그,장애인편의시설,구
0,한국어,63스퀘어,150-763 서울 영등포구 여의도동 60,"07345 서울 영등포구 63로 50 (여의도동, 63한화생명빌딩)",1833-7001,www.63art.co.kr,매일 10:00 - 19:00 아쿠아리움 / 연중무휴 (운영시간 마감 30분 전 입...,월~일,연중무휴,1호선 대방역 6번 출구 셔틀버스 탑승 \r\n5호선 여의도역 5번 출구 셔틀버...,"여의도, 아쿠아리움, 전망대, 데이트코스, 63아쿠아플라넷, 63스퀘어, 63빌딩레...","엘리베이터,장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실,접근가능",영등포구
1,영어,Bongeunsa Temple,"135-870 73, Samseong-dong, Gangnam-gu, Seoul, ...","06087 531, Bongeunsa-ro, Gangnam-gu, Seoul, Ko...",+82-2-3218-4800,http://www.bongeunsa.org/,,,,"* Subway Line 9, Bongeunsa Station, Exit 1\r\n...","Gangnam,Coex,HistoricSite,BuddhistTemple,Lante...",Accessible Information Centers &amp; Services(...,Gangnam-gu
2,영어,Changdeokgung Palace,"110-360 2-71, Waryong-dong, Jongno-gu, Seoul","03072 2-71, Waryong-dong, Jongno-gu, Seoul",+82-2-762-8261,http://www.cdg.go.kr/eng/,"February - May, 09:00 - 18:00 (Last entry at 1...",,Closed Mondays,"Subway Line 3, Anguk Station, Exit 3 (5 minute...","ChangdeokgungPalace,SeoulHanbokTour,KoreanTrad...",Accessible Information Centers &amp; Services(...,Jongno-gu
3,영어,Changgyeonggung Palace,"110-360 Waryong-dong, Jongno-gu, Seoul 2-1","03072 185, Changgyeonggung-ro, Jongno-gu, Seoul",+82-2-762-4868,http://english.cha.go.kr/html/HtmlPage.do?pg=/...,Daily 09:00 - 21:00 (Last entry at 20:00) *Clo...,,,"Subway Line 4, Hyehwa Station, Exit 4 (15 minu...","Traditional,KoreanHistoricSites,SeoulNightView...",Accessible Information Centers &amp; Services(...,Jongno-gu
4,영어,Culture Station Seoul 284,"100-162 122-28, Bongnae-dong 2(i)-ga, Jung-gu,...","04509 1, Tongil-ro, Jung-gu, Seoul",+82-2-3407-3500,http://www.seoul284.org,Daily 10:00 - 19:00 (Last entry at 18:00)\r\n*...,,"Closed Mondays, New Year, Lunar New Year & Chu...","Subway Lines 1 & 4, Seoul Station, Exit 2","SeoulStation,TrainStation,Exhibition,FormerSeo...",Accessible Information Centers &amp; Services(...,Jung-gu
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,한국어,종묘,110-400 서울 종로구 훈정동 1-2,"03135 서울 종로구 종로 157 (훈정동, 종묘공원)",02-765-0195,http://jm.cha.go.kr/agapp/main/index.do?siteCd=JM,"2월 ~ 5월, 9월 ~ 10월 09:00~18:00 (입장마감 17:00)\r\n...",,화요일,5호선 종로3가역 8번 출구 도보 5분 <br/>\r\n1호선 종로3가역 11번 출...,"창덕궁,종로3가역,종묘제례, 종묘사직,유네스코세계문화유산, 광화문, 종묘, 창경궁","장애인 안내(휠체어 대여 및 기타),장애인화장실,접근가능",종로구
153,한국어,창경궁,110-360 서울 종로구 와룡동 2-1,"03072 서울 종로구 창경궁로 185 (와룡동, 창경궁)",02-762-4868,http://cgg.cha.go.kr/agapp/public/html/HtmlPag...,09:00 ~ 21:00 (입장마감 20:00),화~일,월,4호선 혜화역 4번 출구 도보 15분 \r\n,"서울야경,서울야간데이트,서울가볼만한곳,서울궁궐,한국역사적명소,한국궁궐,한국전통문화,...","장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실",종로구
154,한국어,창경궁 대온실,서울 종로구 와룡동 2-1,"03072 서울 종로구 창경궁로 185 (와룡동, 창경궁)",(02) 762-4868~9,http://cgg.cha.go.kr/agapp/main/index.do?siteC...,,,,혜화역 4번출구 700m,"창경궁대온실,창경궁데이트,창경궁포토존,서울가볼만한곳,창경궁",장애인 안내(휠체어 대여 및 기타),종로구
155,한국어,창덕궁,110-360 서울 종로구 와룡동 2-71,"03072 서울 종로구 율곡로 99 (와룡동, 창덕궁)",02-3668-2300,http://www.cdg.go.kr,2-5월\t09:00 ~ 18:00(입장마감은 17:00)\r\n6-8월\t09:0...,화~일,월,5호선 종로3가역 6번 출구 도보 10분 \r\n3호선 안국역 3번 출구 도보 ...,"창덕궁,서울데이트코스,한국전통,서울필수여행지,서울여행코스,서울가볼만한곳,한국문화,세...","장애인 안내(휠체어 대여 및 기타),장애인 전용 주차장,장애인화장실",종로구


In [ ]:
seoul_attraction_df.to_excel('/content/drive/MyDrive/빅데이터통계분석/data/seoul_attraction.xlsx')